In [ ]:
import os
import pandas as pd
from sentence_transformers import SentenceTransformer
import numpy as np
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor

In [1]:
import os
import pandas as pd
import numpy as np
import torch
from sentence_transformers import SentenceTransformer
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor
from tqdm import tqdm

# ✅ 使用 GPU（如果可用）
device = "cuda" if torch.cuda.is_available() else "cpu"
model = SentenceTransformer('DMetaSoul/sbert-chinese-general-v2', device=device)

/Users/Anaconda/anaconda3/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2025-02-14 22:19:40.515782: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
import os
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

# 定义文件路径
root_folder = '/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/data/raw_data_tales'
output_base_folder = '/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/250214similarity_scores'
stories_file = '/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/250211synthetic_archetype_5/250211generated_synthetic_archetypes/250211generated_synthetic_archetypes_1.xlsx'

# 遍历目录，收集所有 .txt 文件路径
all_files = []
for dirpath, _, filenames in os.walk(root_folder):
    for filename in filenames:
        if filename.endswith('.txt'):  # 仅处理 .txt 文件
            relative_path = os.path.join(*dirpath.split(os.sep)[-2:], filename)
            all_files.append((os.path.join(dirpath, filename), relative_path))

# 加载模型
model = SentenceTransformer('DMetaSoul/sbert-chinese-general-v2')

# 加载生成的伪故事
stories_df = pd.read_excel(stories_file)

# 读取文件内容的函数
def read_file(file_info):
    file_path, relative_path = file_info
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            return relative_path, f.read()
    except Exception as e:
        print(f"Error reading file {file_path}: {e}")
        return relative_path, None

# 文件相似度计算函数
def compute_similarity(file_content, story_content):
    if file_content is None:
        return None
    embeddings = model.encode([file_content, story_content])
    similarity = np.dot(embeddings[0], embeddings[1]) / (np.linalg.norm(embeddings[0]) * np.linalg.norm(embeddings[1]))
    return similarity

# 批量处理文件
def process_files_in_batches(all_files, story_content, folder_name, output_path, batch_size=1000):
    batch_number = 1
    os.makedirs(output_path, exist_ok=True)

    for i in range(0, len(all_files), batch_size):
        batch_files = all_files[i:i + batch_size]

        # 读取文件内容
        with ThreadPoolExecutor(max_workers=45) as executor:
            file_contents = list(tqdm(executor.map(read_file, batch_files), total=len(batch_files), desc="Reading files"))

        # 计算相似度
        with ThreadPoolExecutor(max_workers=45) as executor:
            similarities = list(tqdm(executor.map(lambda content: compute_similarity(content[1], story_content), 
                                                  file_contents), total=len(file_contents), desc="Computing similarity"))

        # 组织结果并保存
        batch_results = []
        for (relative_path, content), similarity in zip(file_contents, similarities):
            if content is not None:
                trimmed_relative_path = os.path.join(*relative_path.split(os.sep)[-2:])
                batch_results.append({
                    "文件路径": trimmed_relative_path,
                    "内容": content,
                    "相似度": similarity
                })
        
        # 存储当前批次的结果
        batch_df = pd.DataFrame(batch_results)
        batch_save_path = os.path.join(output_path, f"250108{folder_name}_batch_{batch_number}_similarity_scores.csv")
        batch_df.to_csv(batch_save_path, index=False, encoding='utf-8-sig')
        print(f"Batch {batch_number} for folder '{folder_name}' saved to {batch_save_path}")
        batch_number += 1

# 遍历每个主题，生成相应的相似度结果
for _, row in stories_df.iterrows():
    theme = row['Theme']
    explanation_column = row['Explanation_Column']
    story_content = row['Story']
    
    # 创建基于 Theme 和 Explanation_Column 的输出路径
    folder_name = f"{theme}_{explanation_column}"
    theme_output_path = os.path.join(output_base_folder, folder_name)

    # 执行批量处理
    process_files_in_batches(all_files, story_content, folder_name, theme_output_path)

Computing similarity: 100%|██████████| 1000/1000 [00:35<00:00, 28.52it/s]


Batch 2 for folder '祖先崇拜（Ancestor Worship）_Specific' saved to /Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/250202similarity_scores/祖先崇拜（Ancestor Worship）_Specific/250108祖先崇拜（Ancestor Worship）_Specific_batch_2_similarity_scores.csv


Computing similarity: 100%|██████████| 1000/1000 [00:34<00:00, 28.73it/s]


Batch 3 for folder '祖先崇拜（Ancestor Worship）_Specific' saved to /Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/250202similarity_scores/祖先崇拜（Ancestor Worship）_Specific/250108祖先崇拜（Ancestor Worship）_Specific_batch_3_similarity_scores.csv


Computing similarity: 100%|██████████| 46/46 [00:01<00:00, 28.09it/s]


...

Batch 2 for folder '暴力（Violence）_Generic' saved to /Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/250202similarity_scores/暴力（Violence）_Generic/250108暴力（Violence）_Generic_batch_2_similarity_scores.csv


Computing similarity: 100%|██████████| 1000/1000 [00:34<00:00, 28.83it/s]


Batch 3 for folder '暴力（Violence）_Generic' saved to /Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/250202similarity_scores/暴力（Violence）_Generic/250108暴力（Violence）_Generic_batch_3_similarity_scores.csv


Computing similarity: 100%|██████████| 46/46 [00:01<00:00, 27.79it/s]

Batch 4 for folder '暴力（Violence）_Generic' saved to /Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/250202similarity_scores/暴力（Violence）_Generic/250108暴力（Violence）_Generic_batch_4_similarity_scores.csv


In [5]:
import os
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

# 定义文件路径
root_folder = '/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/data/raw_data_tales'
output_base_folder = '/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/250214similarity_scores'
stories_file = '/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/250211synthetic_archetype_5/250211generated_synthetic_archetypes/250211generated_synthetic_archetypes_1.xlsx'

# 遍历目录，收集所有 .txt 文件路径
all_files = []
for dirpath, _, filenames in os.walk(root_folder):
    for filename in filenames:
        if filename.endswith('.txt'):
            relative_path = os.path.join(*dirpath.split(os.sep)[-2:], filename)
            all_files.append((os.path.join(dirpath, filename), relative_path))

# 加载模型
model = SentenceTransformer('DMetaSoul/sbert-chinese-general-v2')

# 加载生成的伪故事
stories_df = pd.read_excel(stories_file)

# 文件读取函数
def read_file(file_info):
    file_path, relative_path = file_info
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            return relative_path, f.read()
    except Exception as e:
        print(f"Error reading file {file_path}: {e}")
        return relative_path, None

# 批量处理文件函数（优化版本）
def process_files_in_batches(all_files, story_content, folder_name, output_path, batch_size=1000):
    batch_number = 1
    os.makedirs(output_path, exist_ok=True)

    # 预先计算故事文本的embedding
    story_embedding = model.encode(story_content, convert_to_numpy=True)
    story_norm = np.linalg.norm(story_embedding)

    for i in range(0, len(all_files), batch_size):
        batch_files = all_files[i:i + batch_size]

        # 并行读取文件内容
        with ThreadPoolExecutor(max_workers=45) as executor:
            file_contents = list(tqdm(executor.map(read_file, batch_files), total=len(batch_files), desc="Reading files"))

        # 过滤掉内容为 None 的文件
        valid_entries = [(rel, content) for rel, content in file_contents if content is not None]
        if not valid_entries:
            continue

        relative_paths, texts = zip(*valid_entries)

        # 批量编码文件内容
        file_embeddings = model.encode(list(texts), batch_size=64, convert_to_numpy=True)
        # 向量化计算相似度
        file_norms = np.linalg.norm(file_embeddings, axis=1)
        similarities = np.dot(file_embeddings, story_embedding) / (file_norms * story_norm)

        # 组织结果数据
        batch_results = []
        for rel, content, sim in zip(relative_paths, texts, similarities):
            trimmed_relative_path = os.path.join(*rel.split(os.sep)[-2:])
            batch_results.append({
                "文件路径": trimmed_relative_path,
                "内容": content,
                "相似度": sim
            })

        # 保存当前批次结果
        batch_df = pd.DataFrame(batch_results)
        batch_save_path = os.path.join(output_path, f"250108{folder_name}_batch_{batch_number}_similarity_scores.csv")
        batch_df.to_csv(batch_save_path, index=False, encoding='utf-8-sig')
        print(f"Batch {batch_number} for folder '{folder_name}' saved to {batch_save_path}")
        batch_number += 1

# 遍历每个主题，生成对应的相似度结果
for _, row in stories_df.iterrows():
    theme = row['Theme']
    explanation_column = row['Explanation_Column']
    story_content = row['Story']
    
    folder_name = f"{theme}_{explanation_column}"
    theme_output_path = os.path.join(output_base_folder, folder_name)
    process_files_in_batches(all_files, story_content, folder_name, theme_output_path)

Reading files: 100%|██████████| 1000/1000 [00:00<00:00, 28709.04it/s]


Batch 1 for folder '信任（Trust）_Generic' saved to /Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/250214similarity_scores/信任（Trust）_Generic/250108信任（Trust）_Generic_batch_1_similarity_scores.csv


Reading files: 100%|██████████| 1000/1000 [00:00<00:00, 13930.91it/s]


Batch 2 for folder '信任（Trust）_Generic' saved to /Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/250214similarity_scores/信任（Trust）_Generic/250108信任（Trust）_Generic_batch_2_similarity_scores.csv


Reading files: 100%|██████████| 1000/1000 [00:00<00:00, 13872.53it/s]


...

Batch 4 for folder '信任（Trust）_Generic' saved to /Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/250214similarity_scores/信任（Trust）_Generic/250108信任（Trust）_Generic_batch_4_similarity_scores.csv


Reading files: 100%|██████████| 1000/1000 [00:00<00:00, 13020.13it/s]


Batch 5 for folder '信任（Trust）_Generic' saved to /Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/250214similarity_scores/信任（Trust）_Generic/250108信任（Trust）_Generic_batch_5_similarity_scores.csv


Reading files: 100%|██████████| 1000/1000 [00:00<00:00, 12935.60it/s]


KeyboardInterrupt: 

In [2]:
import os
import pandas as pd
from tqdm import tqdm

# 合并每个主题的相似度结果为单独的 CSV 文件
def merge_theme_similarity_files(output_base_folder):
    # 创建存储合并结果的文件夹
    merged_folder = os.path.join(output_base_folder, '250202similarity_scores_merged')
    os.makedirs(merged_folder, exist_ok=True)

    # 遍历输出文件夹，收集所有主题文件夹
    for theme_folder in os.listdir(output_base_folder):
        theme_path = os.path.join(output_base_folder, theme_folder)
        if os.path.isdir(theme_path):  # 确保是文件夹
            # 初始化一个空的 DataFrame 来存储该主题的所有 CSV 数据
            theme_combined_df = pd.DataFrame()
            csv_files = [file for file in os.listdir(theme_path) if file.endswith('_similarity_scores.csv')]

            # 显示进度条
            for file in tqdm(csv_files, desc=f"Merging theme '{theme_folder}'", unit='file'):
                file_path = os.path.join(theme_path, file)
                # 读取 CSV 文件并附加到主题 DataFrame
                df = pd.read_csv(file_path)
                theme_combined_df = pd.concat([theme_combined_df, df], ignore_index=True)

            # 保存合并后的结果为单独的文件到新文件夹
            if not theme_combined_df.empty:
                theme_output_file = os.path.join(merged_folder, f"250202{theme_folder}_similarity_scores_merged.csv")
                theme_combined_df.to_csv(theme_output_file, index=False, encoding='utf-8-sig')
                print(f"主题 '{theme_folder}' 的相似度结果已合并，文件保存为：{theme_output_file}")

# 调用合并函数
merge_theme_similarity_files(output_base_folder)

Merging theme '宗族（Clan）_Generic': 100%|██████████| 4/4 [00:00<00:00, 25.59file/s]


主题 '宗族（Clan）_Generic' 的相似度结果已合并，文件保存为：/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/250202similarity_scores/250202similarity_scores_merged/250202宗族（Clan）_Generic_similarity_scores_merged.csv


Merging theme '暴力（Violence）_Generic': 100%|██████████| 4/4 [00:00<00:00, 27.99file/s]


主题 '暴力（Violence）_Generic' 的相似度结果已合并，文件保存为：/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/250202similarity_scores/250202similarity_scores_merged/250202暴力（Violence）_Generic_similarity_scores_merged.csv


Merging theme '家庭纽带（Family Ties）_Generic': 100%|██████████| 4/4 [00:00<00:00, 28.15file/s]

...

主题 '诚实（Honesty）_Generic' 的相似度结果已合并，文件保存为：/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/250202similarity_scores/250202similarity_scores_merged/250202诚实（Honesty）_Generic_similarity_scores_merged.csv


Merging theme '规则遵循（Rule-Following）_Generic': 100%|██████████| 4/4 [00:00<00:00, 28.33file/s]

主题 '规则遵循（Rule-Following）_Generic' 的相似度结果已合并，文件保存为：/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/250202similarity_scores/250202similarity_scores_merged/250202规则遵循（Rule-Following）_Generic_similarity_scores_merged.csv

Merging theme '慈善（Charity）_Specific_正面': 100%|██████████| 4/4 [00:00<00:00, 28.07file/s]

主题 '慈善（Charity）_Specific_正面' 的相似度结果已合并，文件保存为：/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/250202similarity_scores/250202similarity_scores_merged/250202慈善（Charity）_Specific_正面_similarity_scores_merged.csv


In [3]:
import re

# 合并相似度列到基础表的函数
def merge_similarity_to_base(base_file, output_base_folder):
    # 加载基础表
    # print(base_file)
    base = pd.read_excel(base_file)

    # 获取合并的相似度结果文件夹
    merged_folder = os.path.join(output_base_folder, '250202similarity_scores_merged')
    # print(merged_folder)
    # 初始化合并后的基础表
    merged_base = base.copy()

    # 遍历合并的相似度文件
    for theme_file in tqdm(os.listdir(merged_folder), desc="Merging similarity scores to base"):
        if theme_file.endswith('_similarity_scores_merged.csv'):
            theme_path = os.path.join(merged_folder, theme_file)
            #print(theme_path)
            # print(theme_path)
            data = pd.read_csv(theme_path)

            # 提取文件名
            data['文件名'] = data['文件路径'].apply(
                lambda x: re.search(r'\/(.*?)\.txt', x).group(1) if re.search(r'\/(.*?)\.txt', x) else None
            )
            # print(theme_file)
            # 重新命名相似度列
            theme_name = theme_file[6:]  # 去掉开头六位
            theme_name = theme_name.rsplit('_similarity_scores_merged', 1)[0]  # 去掉结尾部分
            # print(theme_name)
            similarity_column_name = f"{theme_name}_相似度"
            data.rename(columns={'相似度': similarity_column_name}, inplace=True)

            # 合并到基础表中
            merged_base = merged_base.merge(data[['文件名', similarity_column_name]], left_on='name', right_on='文件名', how='left')

            # 删除临时列
            merged_base.drop(columns=['文件名'], inplace=True)

    return merged_base

# 调用合并函数
output_base_folder = '/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/250202similarity_scores'
base_file = '/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/250203地图匹配_spatial_join_2010_no_inconsistency.xlsx'

# 2. 将合并的相似度列合并到基础表
merged_base = merge_similarity_to_base(base_file, output_base_folder)

# 保存最终合并的基础表
output_file = '/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/250202similarity_scores.csv'  # 替换为保存文件的路径
merged_base.to_csv(output_file, index=False, encoding='utf-8-sig')
print(f"合并后的基础表已保存为：{output_file}")

Merging similarity scores to base: 100%|██████████| 54/54 [00:07<00:00,  7.02it/s]


合并后的基础表已保存为：/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/250202similarity_scores.csv


In [33]:
import pandas as pd

# 文件路径
similarity_scores_path = '/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/250108similarity_scores.csv'
spatial_join_path = "/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/to_use/241202spatial_join_2010.xlsx"
cgss_data_path = "/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/to_use/CGSS2010-2015_withoutvarlabel.dta"

# 读取文件到DataFrame
bert_similarity_scores_df = pd.read_csv(similarity_scores_path)
spatial_join_df = pd.read_excel(spatial_join_path)
cgss_data_df= pd.read_stata(cgss_data_path)

In [34]:
bert_similarity_scores_df.columns

Index(['rowid', 'name', 'cailu', 'provgb', 'citygb', 'cntygb', 'area',
       'geocode', 'PROVGB_1953', 'CITYGB_1953', 'CNTYGB_1953', 'PROVGB_1964',
       'CITYGB_1964', 'CNTYGB_1964', 'PROVGB_1982', 'CITYGB_1982',
       'CNTYGB_1982', 'PROVGB_1990a', 'CITYGB_1990a', 'CNTYGB_1990a',
       'PROVGB_2000', 'CITYGB_2000', 'CNTYGB_2000', 'PROVGB_2010',
       'CITYGB_2010', 'CNTYGB_2010', 'new_name', 'area_group', 'centroid_1982',
       'centroid_1990', 'centroid_2000', 'combined_centroid', 'matched_gbcode',
       '规则遵循（Rule-Following）_Specific_正面_相似度', '信任（Trust）_Generic_相似度',
       '宗族（Clan）_Generic_相似度', '平等（Equality）_Specific_中立_相似度',
       '诚实（Honesty）_Specific_反面_相似度', '家庭纽带（Family Ties）_Specific_相似度',
       '儒家（Confucianism）_Generic_相似度', '慈善（Charity）_Specific_正面_相似度',
       '馈赠（Gift-Giving）_Generic_相似度', '长寿（Longevity）_Generic_相似度',
       '帝王（Emperors）_Generic_相似度', '诚实（Honesty）_Specific_中立_相似度',
       '平等（Equality）_Specific_反面_相似度',
       '儒家中的仁义（Benevolence and Justice

In [35]:
import pandas as pd
# 筛选列名包含“相似度”的列
similarity_columns = [col for col in bert_similarity_scores_df.columns if '相似度' in col]
# 按 CNTYGB_2010_y 分组并计算包含“相似度”列的平均值
grouped_mean = bert_similarity_scores_df.groupby('CNTYGB_2010')[similarity_columns].mean()
grouped_mean

,规则遵循（Rule-Following）_Specific_正面_相似度,信任（Trust）_Generic_相似度,宗族（Clan）_Generic_相似度,平等（Equality）_Specific_中立_相似度,诚实（Honesty）_Specific_反面_相似度,家庭纽带（Family Ties）_Specific_相似度,儒家（Confucianism）_Generic_相似度,慈善（Charity）_Specific_正面_相似度,馈赠（Gift-Giving）_Generic_相似度,长寿（Longevity）_Generic_相似度,...,回顾性视角（Backward-Looking Perspective）_Generic_相似度,宗族（Clan）_Specific_相似度,帝王（Emperors）_Specific_相似度,慈善（Charity）_Generic_相似度,儒家中的孝道（Filial Piety in Confucianism）_Generic_相似度,慈善（Charity）_Specific_反面_相似度,鬼神（Spirits and Deities）_Generic_相似度,佛教（Buddhism）_Specific_相似度,规则遵循（Rule-Following）_Specific_中立_相似度,平等（Equality）_Specific_正面_相似度
CNTYGB_2010,,,,,,,,,,,,,,,,,,,,,
110101.0,0.693669,0.689037,0.677550,0.681648,0.687689,0.679929,0.654873,0.642235,0.718348,0.691354,...,0.676335,0.665025,0.682155,0.681010,0.631417,0.648439,0.691263,0.666983,0.703684,0.619692
110102.0,0.694088,0.692399,0.688139,0.685772,0.683743,0.693405,0.660171,0.659001,0.723015,0.704670,...,0.675210,0.648872,0.667515,0.691763,0.644656,0.667948,0.689054,0.686837,0.710074,0.623568
110103.0,0.675598,0.690344,0.676150,0.679599,0.678446,0.685001,0.662480,0.660149,0.721540,0.688712,...,0.644718,0.646097,0.643838,0.693388,0.651957,0.666936,0.669020,0.676592,0.697597,0.638531
110104.0,0.686723,0.703132,0.680675,0.680892,0.689792,0.691175,0.678063,0.668907,0.719964,0.702806,...,0.652305,0.656039,0.675243,0.695235,0.650681,0.673213,0.674190,0.674201,0.708620,0.642534
110105.0,0.721604,0.719428,0.698556,0.714636,0.698519,0.715540,0.684137,0.692003,0.737436,0.739319,...,0.694979,0.667701,0.661397,0.718864,0.677642,0.689805,0.724732,0.704376,0.736459,0.651165
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
654322.0,0.689676,0.715872,0.682992,0.697813,0.649545,0.722225,0.670779,0.688378,0.691909,0.704036,...,0.671751,0.637731,0.660240,0.693668,0.669422,0.676831,0.712349,0.714185,0.698105,0.645308
654323.0,0.661726,0.688718,0.706831,0.677450,0.678581,0.689411,0.672605,0.647748,0.656756,0.706992,...,0.636006,0.683590,0.681297,0.649749,0.622921,0.652240,0.689293,0.664385,0.666994,0.637612
654324.0,0.693598,0.721122,0.698729,0.706240,0.653347,0.736984,0.678674,0.689523,0.726155,0.712042,...,0.661023,0.666074,0.653307,0.712421,0.684642,0.691103,0.709314,0.706534,0.708426,0.653234


In [36]:
# 替换列名为统一格式，去掉结尾的“_相似度”
grouped_mean.rename(
    columns=lambda x: f"bert_similarity_{x.rsplit('_相似度', 1)[0]}" if x.endswith('_相似度') else x,
    inplace=True
)

grouped_mean

,bert_similarity_规则遵循（Rule-Following）_Specific_正面,bert_similarity_信任（Trust）_Generic,bert_similarity_宗族（Clan）_Generic,bert_similarity_平等（Equality）_Specific_中立,bert_similarity_诚实（Honesty）_Specific_反面,bert_similarity_家庭纽带（Family Ties）_Specific,bert_similarity_儒家（Confucianism）_Generic,bert_similarity_慈善（Charity）_Specific_正面,bert_similarity_馈赠（Gift-Giving）_Generic,bert_similarity_长寿（Longevity）_Generic,...,bert_similarity_回顾性视角（Backward-Looking Perspective）_Generic,bert_similarity_宗族（Clan）_Specific,bert_similarity_帝王（Emperors）_Specific,bert_similarity_慈善（Charity）_Generic,bert_similarity_儒家中的孝道（Filial Piety in Confucianism）_Generic,bert_similarity_慈善（Charity）_Specific_反面,bert_similarity_鬼神（Spirits and Deities）_Generic,bert_similarity_佛教（Buddhism）_Specific,bert_similarity_规则遵循（Rule-Following）_Specific_中立,bert_similarity_平等（Equality）_Specific_正面
CNTYGB_2010,,,,,,,,,,,,,,,,,,,,,
110101.0,0.693669,0.689037,0.677550,0.681648,0.687689,0.679929,0.654873,0.642235,0.718348,0.691354,...,0.676335,0.665025,0.682155,0.681010,0.631417,0.648439,0.691263,0.666983,0.703684,0.619692
110102.0,0.694088,0.692399,0.688139,0.685772,0.683743,0.693405,0.660171,0.659001,0.723015,0.704670,...,0.675210,0.648872,0.667515,0.691763,0.644656,0.667948,0.689054,0.686837,0.710074,0.623568
110103.0,0.675598,0.690344,0.676150,0.679599,0.678446,0.685001,0.662480,0.660149,0.721540,0.688712,...,0.644718,0.646097,0.643838,0.693388,0.651957,0.666936,0.669020,0.676592,0.697597,0.638531
110104.0,0.686723,0.703132,0.680675,0.680892,0.689792,0.691175,0.678063,0.668907,0.719964,0.702806,...,0.652305,0.656039,0.675243,0.695235,0.650681,0.673213,0.674190,0.674201,0.708620,0.642534
110105.0,0.721604,0.719428,0.698556,0.714636,0.698519,0.715540,0.684137,0.692003,0.737436,0.739319,...,0.694979,0.667701,0.661397,0.718864,0.677642,0.689805,0.724732,0.704376,0.736459,0.651165
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
654322.0,0.689676,0.715872,0.682992,0.697813,0.649545,0.722225,0.670779,0.688378,0.691909,0.704036,...,0.671751,0.637731,0.660240,0.693668,0.669422,0.676831,0.712349,0.714185,0.698105,0.645308
654323.0,0.661726,0.688718,0.706831,0.677450,0.678581,0.689411,0.672605,0.647748,0.656756,0.706992,...,0.636006,0.683590,0.681297,0.649749,0.622921,0.652240,0.689293,0.664385,0.666994,0.637612
654324.0,0.693598,0.721122,0.698729,0.706240,0.653347,0.736984,0.678674,0.689523,0.726155,0.712042,...,0.661023,0.666074,0.653307,0.712421,0.684642,0.691103,0.709314,0.706534,0.708426,0.653234


In [37]:
grouped_mean.reset_index(inplace=True)
if 'CNTYGB_2010' in grouped_mean.columns:
    # 将CNTYGB_2010_y列转换为整数类型，处理NaN值为默认值（如0）
    grouped_mean['CNTYGB_2010'] = grouped_mean['CNTYGB_2010'].fillna(0).astype(int)
else:
    print("列名 'CNTYGB_2010' 不存在，请检查数据框")
grouped_mean

,CNTYGB_2010,bert_similarity_规则遵循（Rule-Following）_Specific_正面,bert_similarity_信任（Trust）_Generic,bert_similarity_宗族（Clan）_Generic,bert_similarity_平等（Equality）_Specific_中立,bert_similarity_诚实（Honesty）_Specific_反面,bert_similarity_家庭纽带（Family Ties）_Specific,bert_similarity_儒家（Confucianism）_Generic,bert_similarity_慈善（Charity）_Specific_正面,bert_similarity_馈赠（Gift-Giving）_Generic,...,bert_similarity_回顾性视角（Backward-Looking Perspective）_Generic,bert_similarity_宗族（Clan）_Specific,bert_similarity_帝王（Emperors）_Specific,bert_similarity_慈善（Charity）_Generic,bert_similarity_儒家中的孝道（Filial Piety in Confucianism）_Generic,bert_similarity_慈善（Charity）_Specific_反面,bert_similarity_鬼神（Spirits and Deities）_Generic,bert_similarity_佛教（Buddhism）_Specific,bert_similarity_规则遵循（Rule-Following）_Specific_中立,bert_similarity_平等（Equality）_Specific_正面
0,110101,0.693669,0.689037,0.677550,0.681648,0.687689,0.679929,0.654873,0.642235,0.718348,...,0.676335,0.665025,0.682155,0.681010,0.631417,0.648439,0.691263,0.666983,0.703684,0.619692
1,110102,0.694088,0.692399,0.688139,0.685772,0.683743,0.693405,0.660171,0.659001,0.723015,...,0.675210,0.648872,0.667515,0.691763,0.644656,0.667948,0.689054,0.686837,0.710074,0.623568
2,110103,0.675598,0.690344,0.676150,0.679599,0.678446,0.685001,0.662480,0.660149,0.721540,...,0.644718,0.646097,0.643838,0.693388,0.651957,0.666936,0.669020,0.676592,0.697597,0.638531
3,110104,0.686723,0.703132,0.680675,0.680892,0.689792,0.691175,0.678063,0.668907,0.719964,...,0.652305,0.656039,0.675243,0.695235,0.650681,0.673213,0.674190,0.674201,0.708620,0.642534
4,110105,0.721604,0.719428,0.698556,0.714636,0.698519,0.715540,0.684137,0.692003,0.737436,...,0.694979,0.667701,0.661397,0.718864,0.677642,0.689805,0.724732,0.704376,0.736459,0.651165
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1716,654322,0.689676,0.715872,0.682992,0.697813,0.649545,0.722225,0.670779,0.688378,0.691909,...,0.671751,0.637731,0.660240,0.693668,0.669422,0.676831,0.712349,0.714185,0.698105,0.645308
1717,654323,0.661726,0.688718,0.706831,0.677450,0.678581,0.689411,0.672605,0.647748,0.656756,...,0.636006,0.683590,0.681297,0.649749,0.622921,0.652240,0.689293,0.664385,0.666994,0.637612
1718,654324,0.693598,0.721122,0.698729,0.706240,0.653347,0.736984,0.678674,0.689523,0.726155,...,0.661023,0.666074,0.653307,0.712421,0.684642,0.691103,0.709314,0.706534,0.708426,0.653234
1719,654326,0.690318,0.691506,0.700173,0.693829,0.648174,0.719580,0.653165,0.716280,0.759801,...,0.666878,0.662321,0.639959,0.743025,0.710068,0.698561,0.677012,0.716972,0.685008,0.650264


In [38]:
# 确保列名存在
if 'china10_cntygb' in cgss_data_df.columns:
    # 将 china10_cntygb 列转换为整数类型，空值填充为 0
    cgss_data_df['china10_cntygb'] = cgss_data_df['china10_cntygb'].fillna(0).astype(int)
else:
    print("列名 'china10_cntygb' 不存在，请检查数据框")
cgss_data_df.head()

,id_old,id_str,surveyyear,id_surveyyear,s41,s42,s43,cntyname,china10_cntygb,s44,...,e112a,e112b,e113,e114,e115,z8,a59disco88,a60disco88,china10_wcd,china10_prefgb_special
0,6132.0,6132,2015,6132_2015,1.0,1.0,1.0,上海市徐汇区,310104,1.0,...,NaN,NaN,NaN,NaN,NaN,2015-10-27 16:15:00,2300.0,NaN,3101200.0,68.0
1,10915.0,10915,2012,10915_2012,1.0,1.0,1.0,上海市徐汇区,310104,1.0,...,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,3101200.0,68.0
2,10328.0,10328,2013,10328_2013,1.0,1.0,1.0,上海市徐汇区,310104,1.0,...,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,3101200.0,68.0
3,11566.0,11566,2010,11566_2010,1.0,1.0,1.0,上海市徐汇区,310104,1.0,...,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,3101200.0,68.0
4,346.0,346,2013,346_2013,1.0,1.0,1.0,上海市徐汇区,310104,1.0,...,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,3101200.0,68.0


In [39]:
# 确保需要合并的列存在
if 'CNTYGB_2010' in grouped_mean.columns and 'china10_cntygb' in cgss_data_df.columns:
    # 按照 CNTYGB_2010_y 和 china10_cntygb 列的值进行合并
    merged_df = cgss_data_df.merge(grouped_mean, left_on='china10_cntygb', right_on='CNTYGB_2010', how='left')
    # 删除 CNTYGB_2010_y 列
    merged_df.drop(columns=['CNTYGB_2010'], inplace=True)
merged_df.head()

,id_old,id_str,surveyyear,id_surveyyear,s41,s42,s43,cntyname,china10_cntygb,s44,...,bert_similarity_回顾性视角（Backward-Looking Perspective）_Generic,bert_similarity_宗族（Clan）_Specific,bert_similarity_帝王（Emperors）_Specific,bert_similarity_慈善（Charity）_Generic,bert_similarity_儒家中的孝道（Filial Piety in Confucianism）_Generic,bert_similarity_慈善（Charity）_Specific_反面,bert_similarity_鬼神（Spirits and Deities）_Generic,bert_similarity_佛教（Buddhism）_Specific,bert_similarity_规则遵循（Rule-Following）_Specific_中立,bert_similarity_平等（Equality）_Specific_正面
0,6132.0,6132,2015,6132_2015,1.0,1.0,1.0,上海市徐汇区,310104,1.0,...,0.65834,0.667419,0.65091,0.709557,0.682354,0.697604,0.694679,0.702501,0.723178,0.652779
1,10915.0,10915,2012,10915_2012,1.0,1.0,1.0,上海市徐汇区,310104,1.0,...,0.65834,0.667419,0.65091,0.709557,0.682354,0.697604,0.694679,0.702501,0.723178,0.652779
2,10328.0,10328,2013,10328_2013,1.0,1.0,1.0,上海市徐汇区,310104,1.0,...,0.65834,0.667419,0.65091,0.709557,0.682354,0.697604,0.694679,0.702501,0.723178,0.652779
3,11566.0,11566,2010,11566_2010,1.0,1.0,1.0,上海市徐汇区,310104,1.0,...,0.65834,0.667419,0.65091,0.709557,0.682354,0.697604,0.694679,0.702501,0.723178,0.652779
4,346.0,346,2013,346_2013,1.0,1.0,1.0,上海市徐汇区,310104,1.0,...,0.65834,0.667419,0.65091,0.709557,0.682354,0.697604,0.694679,0.702501,0.723178,0.652779


In [40]:
# 定义保存路径
output_csv_path = '/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/250108cgss_bert_county_level_mean.csv'
# 保存为 CSV 文件
merged_df.to_csv(output_csv_path, index=False, encoding='utf-8-sig')
print(f"合并后的数据已保存到: {output_csv_path}")

合并后的数据已保存到: /Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/250108cgss_bert_county_level_mean.csv
